# 🌳 Arbor in Google Colab - Quick Start

This notebook demonstrates how to use Arbor in Google Colab for fine-tuning language models with a Ray-like interface.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ziems/arbor/blob/main/examples/colab_quickstart.ipynb)

## 📦 Installation

First, let's install Arbor using `uv` for faster package management. This may take a few minutes as it installs the ML dependencies, but uv is significantly faster than pip.

In [ ]:
# Install uv for faster package management
!curl -LsSf https://astral.sh/uv/install.sh | sh
import os
os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

# Install Arbor from the colab-support branch (includes new Colab integration)
!uv pip install --system git+https://github.com/Ziems/arbor.git@colab-support openai

# Optional: Install flash attention for faster inference (takes ~15 min)
# !uv pip install --system flash-attn --no-build-isolation

## 🚀 Start Arbor Server

Start Arbor in the background with a Ray-like interface. This will:
- Auto-detect available GPUs
- Create a config file automatically
- Start the server in a background thread
- Provide an OpenAI-compatible API

In [ ]:
import arbor

# Start Arbor (Ray-like interface)
server_info = arbor.init()
print(f"Server running at: {server_info['base_url']}")

## 🔌 Connect with OpenAI Client

Use the standard OpenAI Python client to interact with Arbor.

In [ ]:
# Option 1: Use the convenience method
client = arbor.get_client()

# Option 2: Manual setup
# from openai import OpenAI
# client = OpenAI(
#     base_url=server_info['base_url'],
#     api_key="not-needed"  # Local server doesn't need API key
# )

## 💬 Test Inference

Let's test inference with a small model to make sure everything works.

In [ ]:
# Test chat completion
response = client.chat.completions.create(
    model="HuggingFaceTB/SmolLM2-135M-Instruct",  # Small model for testing
    messages=[
        {"role": "user", "content": "Hello! Tell me a joke about machine learning."}
    ],
    temperature=0.7,
    max_tokens=100
)

print("🤖 Response:", response.choices[0].message.content)

## 📁 Upload Training Data

Let's create some sample training data and upload it.

In [ ]:
import json
import tempfile
import os

# Create sample SFT training data
sample_data = [
    {
        "messages": [
            {"role": "user", "content": "What is machine learning?"},
            {"role": "assistant", "content": "Machine learning is a subset of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Explain neural networks simply."},
            {"role": "assistant", "content": "Neural networks are computing systems inspired by biological neural networks. They consist of interconnected nodes (neurons) that process information and learn patterns from data."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "What is deep learning?"},
            {"role": "assistant", "content": "Deep learning is a type of machine learning that uses neural networks with multiple layers to model and understand complex patterns in data."}
        ]
    }
]

# Save to temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.jsonl', delete=False) as f:
    for item in sample_data:
        f.write(json.dumps(item) + '\n')
    temp_file = f.name

print(f"Created training data: {temp_file}")

In [ ]:
# Upload the training file
with open(temp_file, 'rb') as f:
    file_response = client.files.create(
        file=f,
        purpose='fine-tune'
    )

print(f"📁 File uploaded: {file_response.id}")
training_file_id = file_response.id

## 🎯 Start Fine-tuning Job

Now let's start a supervised fine-tuning (SFT) job.

In [ ]:
# Create fine-tuning job
job = client.fine_tuning.jobs.create(
    method={"type": "sft"},
    training_file=training_file_id,
    model="HuggingFaceTB/SmolLM2-135M-Instruct"  # Small model for quick training
)

print(f"🎯 Training job created: {job.id}")
print(f"Status: {job.status}")
job_id = job.id

## 📊 Monitor Training Progress with Real-time Logs

Arbor now provides built-in log monitoring! You can watch training progress in real-time.

In [ ]:
# Option 1: Watch job with real-time logs (recommended!)
print("🎬 Starting real-time job monitoring...")
arbor.watch_job(job_id, max_time=300, show_logs=True)  # Watch for 5 minutes max

In [ ]:
# Option 2: Traditional polling approach
import time

# Check job status manually
job_status = client.fine_tuning.jobs.retrieve(job_id)
print(f"📊 Job Status: {job_status.status}")

# Monitor until completion (with timeout)
timeout = 300  # 5 minutes
start_time = time.time()

while job_status.status in ['queued', 'running'] and (time.time() - start_time) < timeout:
    time.sleep(10)  # Check every 10 seconds
    job_status = client.fine_tuning.jobs.retrieve(job_id)
    elapsed = int(time.time() - start_time)
    print(f"⏱️  [{elapsed}s] Status: {job_status.status}")

print(f"\n🎉 Final Status: {job_status.status}")
if job_status.fine_tuned_model:
    print(f"✅ Fine-tuned model: {job_status.fine_tuned_model}")

### 🆚 Monitoring Options

**Option 1: `arbor.watch_job()`** - Shows real-time training logs as they happen ✨  
**Option 2: Traditional polling** - Just checks status periodically

The new `watch_job()` function gives you much better visibility into what's happening during training!

## 🧪 Test Fine-tuned Model

If training completed successfully, let's test the fine-tuned model.

In [ ]:
if job_status.status == 'succeeded' and job_status.fine_tuned_model:
    # Test the fine-tuned model
    response = client.chat.completions.create(
        model=job_status.fine_tuned_model,
        messages=[
            {"role": "user", "content": "What is reinforcement learning?"}
        ],
        temperature=0.7,
        max_tokens=150
    )
    
    print("🎭 Fine-tuned model response:")
    print(response.choices[0].message.content)
else:
    print(f"⚠️  Training not completed yet. Status: {job_status.status}")
    if job_status.status == 'failed':
        print("❌ Training failed. Check the logs for details.")

## 🚀 Advanced: GRPO Training

For more advanced users, here's how to do GRPO (Generalized Reward-based Policy Optimization) training.

In [ ]:
# GRPO training example (commented out by default)
# Uncomment and modify as needed

# grpo_job = client.fine_tuning.jobs.create(
#     method={
#         "type": "grpo",
#         "hyperparameters": {
#             "num_train_epochs": 1,
#             "learning_rate": 5e-5,
#             "per_device_train_batch_size": 1
#         }
#     },
#     training_file=training_file_id,
#     model="HuggingFaceTB/SmolLM2-135M-Instruct"
# )
# 
# print(f"🎯 GRPO job created: {grpo_job.id}")

print("💡 GRPO example is commented out. Uncomment to try it!")

## 🛠️ Server Management

Arbor provides utilities to check server status and manage the background process.

In [ ]:
# Check server status
status = arbor.status()
if status:
    print("🌳 Arbor server is running:")
    print(f"   Host: {status['host']}")
    print(f"   Port: {status['port']}")
    print(f"   Base URL: {status['base_url']}")
    print(f"   Storage: {status['storage_path']}")
    print(f"   GPU IDs: {status['gpu_ids']}")
else:
    print("❌ No Arbor server running")

## 🧹 Cleanup

When you're done, you can shut down the Arbor server (optional - it will auto-cleanup when the notebook ends).

In [ ]:
# Shutdown the server (optional)
# arbor.shutdown()

# Clean up temporary file
if 'temp_file' in locals():
    os.unlink(temp_file)
    print("🗑️  Cleaned up temporary training file")

print("✅ Session cleanup complete")

## 🎯 Next Steps

Now that you've got Arbor running in Colab, here are some things to try:

1. **Experiment with different models**: Try larger models like `Qwen/Qwen2-0.5B-Instruct`
2. **Use your own data**: Upload your own JSONL training files
3. **Try GRPO training**: Uncomment the GRPO example above
4. **Integrate with DSPy**: Use Arbor as a backend for DSPy programs
5. **Monitor training**: Use the monitoring endpoints to track GPU usage and training metrics

### 📚 Resources

- [Arbor GitHub Repository](https://github.com/Ziems/arbor)
- [DSPy RL Optimization Examples](https://dspy.ai/tutorials/rl_papillon/)
- [DSPy Discord Community](https://discord.gg/ZAEGgxjPUe)

### 💡 Pro Tips for Colab

- Use `arbor.init()` at the start of your notebook - it's designed to be safe to call multiple times
- The server will auto-cleanup when your Colab session ends
- For long-running training jobs, consider upgrading to Colab Pro for more stable GPU access
- Monitor your GPU usage to avoid hitting Colab limits

Happy fine-tuning! 🌳✨